In [2]:
from herbie import Herbie
from datetime import datetime, timedelta
import math
from herbie import FastHerbie
import pandas as pd
    
start_time_str = '2023-11-06 00:00:00'
startrun = 23 # first forecast hour in run, does not have to start at 0
maxrun = 25 # last forecast hour in run, to get 48hr model: maxrun = 49

file_name_list = ['/home/bran/SNOWPACK/input/CUES/CUES_test.smet', '/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet', '/home/bran/SNOWPACK/input/Rock_Creek/Rock_Creek_test.smet']
ll = [(-119.028661,37.645588),(-119.256899,37.935258),(-118.726413,37.452796)] # lat/long for selected HRRR grid points. lat/long order: CUES, Tioga, Rock_Creek
ss1 = ":TMP:2 m|:RH:2 m" # 2m readings
ss2 = ":WIND:10 m" # 10m readings
ss3 = ":GUST:surface|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface:(?:0-1|[1-9]\d*-\d+)*" # surf readings minus precip


DATES = pd.date_range(
    start = start_time_str,
    periods=1,
    freq="24H")

fxx = range(startrun,maxrun) 

FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
FH
print('querying data for 2 m')
ds1 = FH.xarray(ss1, remove_grib=False)
y1=ds1.herbie.nearest_points(points=ll)
y1_d = y1.to_dict()

print('querying data for 10 m')
ds2 = FH.xarray(ss2, remove_grib=False)
y2=ds2.herbie.nearest_points(points=ll)
y2_d = y2.to_dict()

print('querying data for surface')
ds3 = FH.xarray(ss3, remove_grib=False)
y3=ds3.herbie.nearest_points(points=ll)
y3_d = y3.to_dict()

for i in range(startrun,maxrun): # iterating data for each hour
    dt = datetime.fromisoformat(start_time_str)
    dt_query_str = dt.strftime('%Y-%m-%d %H:%M')
    output_timestamp = dt + timedelta(hours=i)
    output_timestamp_str = output_timestamp.strftime('%Y-%m-%dT%H:%M')
    hours_offset = i
    
    for j in range(3): # printing to smet for each station, range should match number of stations, 0->n
        data_index = j
        append_data(y1_d, y2_d, y3_d, output_timestamp_str)
        
print("done")

querying data for 2 m


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "t2m".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "r2".
  ds = ds.metpy.assign_y_x()


querying data for 10 m


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "si10".
  ds = ds.metpy.assign_y_x()


querying data for surface
Ignore This
Ignore This
Ignore This
<class 'list'>
209
210
<class 'list'>
209
210
<class 'list'>
209
210
done


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "tp".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gust".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "dswrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "uswrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "dlwrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordin

In [1]:
def append_data(y1_d, y2_d, y3_d, output_timestamp_str):
    
    timeindex = i - startrun
    t2m = y1_d['data_vars']['t2m']['data'][data_index][timeindex]
    r2 = y1_d['data_vars']['r2']['data'][data_index][timeindex]
    wind10 = y2_d['data_vars']['si10']['data'][data_index][timeindex]
    gust = y3_d['data_vars']['gust']['data'][data_index][timeindex]
    dsw = y3_d['data_vars']['dswrf']['data'][data_index][timeindex]
    dlw = y3_d['data_vars']['dlwrf']['data'][data_index][timeindex]
    ulw = y3_d['data_vars']['ulwrf']['data'][data_index][timeindex]
    usw = y3_d['data_vars']['uswrf']['data'][data_index][timeindex]
    hrprecip = y3_d['data_vars']['tp']['data'][data_index][timeindex]
    if i==0:
        hrpreciplast = 0
    else:
        hrpreciplast = y3_d['data_vars']['tp']['data'][data_index][timeindex-1]
    hrprecipfile = hrprecip - hrpreciplast
        
    t2m_formatted = round(t2m, 1) # 2nd parameter is number of decimals
    r2_formatted = round(r2)
    gust_formatted = round(gust, 1)
    wind10_formatted = round(wind10, 1)
    dsw_formatted = round(dsw)
    dlw_formatted = round(dlw)
    ulw_formatted = round(ulw)
    usw_formatted = round(usw)
    hrprecip_formatted = round(hrprecipfile, 1)
    
    file_name = file_name_list[j]
    
    if i == startrun:
        print("Ignore This")
        
    else:
    
        new_line_str = f"{output_timestamp_str} {t2m_formatted} {r2_formatted} {gust_formatted} {wind10_formatted} {dsw_formatted} {dlw_formatted} {ulw_formatted} {usw_formatted} {hrprecip_formatted} 0.0\n"
        with open(file_name, 'r') as f:
            lines = f.readlines()
            print(type(lines))
            print(len(lines))
            new_lines = lines
            found= False
            for idx, line in enumerate(lines):
                if line[:16] == output_timestamp_str:
                    new_lines[idx] = new_line_str
                    found = True
                    break
            if not found:
                new_lines += [new_line_str]
        print(len(new_lines))
        with open(file_name, 'w') as f:
            f.writelines(new_lines)
        
print("done")

done


In [5]:
print(ds3)

<xarray.Dataset>
Dimensions:              (step: 2, y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2023-10-25
  * step                 (step) timedelta64[ns] 00:00:00 01:00:00
    surface              float64 0.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           (step) datetime64[ns] 2023-10-25 2023-10-25T01:00:00
Dimensions without coordinates: y, x
Data variables:
    tp                   (step, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    gust                 (step, y, x) float32 5.435 5.373 5.373 ... 11.6 11.91
    dswrf                (step, y, x) float32 276.7 283.7 301.2 ... 0.0 0.0 0.0
    uswrf                (step, y, x) float32 22.12 22.75 24.12 ... 0.0 0.0 0.0
    dlwrf                (step, y, x) float32 355.0 353.0 345.3 ... 271.3 271.1
    ulwrf                (step, y, x) float32 429.9 429.8 429.8 ... 361.3 3

In [12]:
FH.inventory(ss3)

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this,FILE
0,9,3933992,5105731,3933992-5105731,2023-10-25,2023-10-25 00:00:00,GUST,surface,anl,:GUST:surface:anl,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
1,84,52291335,52291547,52291335-52291547,2023-10-25,2023-10-25 00:00:00,APCP,surface,0-0 day acc fcst,:APCP:surface:0-0 day acc fcst,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
2,123,80178708,80970867,80178708-80970867,2023-10-25,2023-10-25 00:00:00,DSWRF,surface,anl,:DSWRF:surface:anl,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
3,124,80970867,83014814,80970867-83014814,2023-10-25,2023-10-25 00:00:00,DLWRF,surface,anl,:DLWRF:surface:anl,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
4,125,83014814,83601864,83014814-83601864,2023-10-25,2023-10-25 00:00:00,USWRF,surface,anl,:USWRF:surface:anl,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
5,126,83601864,85220008,83601864-85220008,2023-10-25,2023-10-25 00:00:00,ULWRF,surface,anl,:ULWRF:surface:anl,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
6,9,3815279,5038050,3815279-5038050,2023-10-25,2023-10-25 01:00:00,GUST,surface,1 hour fcst,:GUST:surface:1 hour fcst,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
7,84,59866975,60294242,59866975-60294242,2023-10-25,2023-10-25 01:00:00,APCP,surface,0-1 hour acc fcst,:APCP:surface:0-1 hour acc fcst,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
8,123,89961430,90290232,89961430-90290232,2023-10-25,2023-10-25 01:00:00,DSWRF,surface,1 hour fcst,:DSWRF:surface:1 hour fcst,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...
9,124,90290232,92302355,90290232-92302355,2023-10-25,2023-10-25 01:00:00,DLWRF,surface,1 hour fcst,:DLWRF:surface:1 hour fcst,https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrr...


In [13]:
print(y3_d)

{'coords': {'time': {'dims': (), 'attrs': {'long_name': 'initial time of forecast', 'standard_name': 'forecast_reference_time', '_metpy_axis': 'time'}, 'data': datetime.datetime(2023, 10, 25, 0, 0)}, 'step': {'dims': ('step',), 'attrs': {'long_name': 'time since forecast_reference_time', 'standard_name': 'forecast_period'}, 'data': [datetime.timedelta(0), datetime.timedelta(seconds=3600), datetime.timedelta(seconds=7200)]}, 'surface': {'dims': (), 'attrs': {'long_name': 'original GRIB coordinate for key: level(surface)', 'units': '1'}, 'data': 0.0}, 'latitude': {'dims': ('point',), 'attrs': {'units': 'degrees_north', 'standard_name': 'latitude', 'long_name': 'latitude', '_metpy_axis': 'latitude'}, 'data': [37.645587779088835, 37.935257636348595, 37.45279612720201]}, 'longitude': {'dims': ('point',), 'attrs': {'units': 'degrees_east', 'standard_name': 'longitude', 'long_name': 'longitude', '_metpy_axis': 'longitude'}, 'data': [240.9713397600786, 240.74311152188147, 241.27359493729398]},

In [23]:
hrprecip1 = y3_d['data_vars']['tp']['data']
print(hrprecip1)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.029999999329447746, 0.5799999833106995, 2.200000047683716, 5.880000114440918, 7.090000152587891], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019999999552965164, 0.36000001430511475, 1.3600000143051147, 2.4800000190734863], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05999999865889549, 0.2199999988079071, 0.550000011920929, 1.0299999713897705, 1.0800000429153442]]
